# Fake news project

## Dask cluster

In [1]:
from dask.distributed import Client, LocalCluster

cluster = LocalCluster()
client = Client(cluster)
client

2023-03-16 15:38:12,612 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-me_q54p5', purging
2023-03-16 15:38:12,774 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-yl43oj87', purging
2023-03-16 15:38:12,913 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-mdnb4fwb', purging
2023-03-16 15:38:13,065 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-ig19lhhz', purging


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 8,Total memory: 15.46 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:46015,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 15.46 GiB
Comm: tcp://127.0.0.1:40131,Total threads: 2
Dashboard: http://127.0.0.1:46593/status,Memory: 3.87 GiB
Nanny: tcp://127.0.0.1:44287,


## Models

### SBERT
To download the SBERT model run the following command:
```
pipenv install sentence-transformers
```

## Data import


In [2]:
# Import the data 
import dask.dataframe as dd
ddf = dd.read_parquet('data/filtered_parquets/')
ddf.head()

,id,content,type,title,authors,domain,url
27,34,Headline: Bitcoin & Blockchain Searches Exceed...,fake,Surprise: Socialist Hotbed Of Venezuela Has Lo...,The Pirate'S Cove,beforeitsnews.com,http://beforeitsnews.com/opinion-conservative/...
28,35,Water Cooler 1/25/18 Open Thread; Fake News ? ...,fake,Water Cooler 1/25/18 Open Thread; Fake News ? ...,,beforeitsnews.com,http://beforeitsnews.com/politics/2018/01/wate...
29,36,Veteran Commentator Calls Out the Growing “Eth...,fake,Veteran Commentator Calls Out the Growing “Eth...,,beforeitsnews.com,http://beforeitsnews.com/politics/2018/01/vete...
30,37,"Lost Words, Hidden Words, Otters, Banks and Bo...",fake,"Lost Words, Hidden Words, Otters, Banks and Books",Jackie Morris Artist,beforeitsnews.com,http://beforeitsnews.com/arts/2018/01/lost-wor...
31,38,Red Alert: Bond Yields Are SCREAMING “Inflatio...,fake,Red Alert: Bond Yields Are SCREAMING “Inflatio...,Phoenix Capital Research,beforeitsnews.com,http://beforeitsnews.com/financial-markets/201...


In [10]:
ddf.type.value_counts().compute()

reliable    1913222
fake         894746
Name: type, dtype: int64

## Data processing

### Split data so it only contains reliable and fake

In [19]:
# ddf = ddf[ddf['type'].isin(['reliable', 'fake'])]
# ddf.head()

,id,content,type,title,authors,domain,url
27,34,Headline: Bitcoin & Blockchain Searches Exceed...,fake,Surprise: Socialist Hotbed Of Venezuela Has Lo...,The Pirate'S Cove,beforeitsnews.com,http://beforeitsnews.com/opinion-conservative/...
28,35,Water Cooler 1/25/18 Open Thread; Fake News ? ...,fake,Water Cooler 1/25/18 Open Thread; Fake News ? ...,,beforeitsnews.com,http://beforeitsnews.com/politics/2018/01/wate...
29,36,Veteran Commentator Calls Out the Growing “Eth...,fake,Veteran Commentator Calls Out the Growing “Eth...,,beforeitsnews.com,http://beforeitsnews.com/politics/2018/01/vete...
30,37,"Lost Words, Hidden Words, Otters, Banks and Bo...",fake,"Lost Words, Hidden Words, Otters, Banks and Books",Jackie Morris Artist,beforeitsnews.com,http://beforeitsnews.com/arts/2018/01/lost-wor...
31,38,Red Alert: Bond Yields Are SCREAMING “Inflatio...,fake,Red Alert: Bond Yields Are SCREAMING “Inflatio...,Phoenix Capital Research,beforeitsnews.com,http://beforeitsnews.com/financial-markets/201...


In [20]:
# Write to parquets
# ddf.to_parquet('data/filtered_parquets/', engine='pyarrow', compression='snappy')

In [21]:
# ddf = dd.read_parquet('data/filtered_parquets/')
# ddf.head()

,id,content,type,title,authors,domain,url
27,34,Headline: Bitcoin & Blockchain Searches Exceed...,fake,Surprise: Socialist Hotbed Of Venezuela Has Lo...,The Pirate'S Cove,beforeitsnews.com,http://beforeitsnews.com/opinion-conservative/...
28,35,Water Cooler 1/25/18 Open Thread; Fake News ? ...,fake,Water Cooler 1/25/18 Open Thread; Fake News ? ...,,beforeitsnews.com,http://beforeitsnews.com/politics/2018/01/wate...
29,36,Veteran Commentator Calls Out the Growing “Eth...,fake,Veteran Commentator Calls Out the Growing “Eth...,,beforeitsnews.com,http://beforeitsnews.com/politics/2018/01/vete...
30,37,"Lost Words, Hidden Words, Otters, Banks and Bo...",fake,"Lost Words, Hidden Words, Otters, Banks and Books",Jackie Morris Artist,beforeitsnews.com,http://beforeitsnews.com/arts/2018/01/lost-wor...
31,38,Red Alert: Bond Yields Are SCREAMING “Inflatio...,fake,Red Alert: Bond Yields Are SCREAMING “Inflatio...,Phoenix Capital Research,beforeitsnews.com,http://beforeitsnews.com/financial-markets/201...


### Isolate desired columns
For now, we choose to just use the columns: id, content and type.

In [3]:
ddf = ddf[['id', 'content', 'type']]
ddf.head()

,id,content,type
27,34,Headline: Bitcoin & Blockchain Searches Exceed...,fake
28,35,Water Cooler 1/25/18 Open Thread; Fake News ? ...,fake
29,36,Veteran Commentator Calls Out the Growing “Eth...,fake
30,37,"Lost Words, Hidden Words, Otters, Banks and Bo...",fake
31,38,Red Alert: Bond Yields Are SCREAMING “Inflatio...,fake


### Tokenization

In [4]:
# Removing newline characters
import re
def remove_newlines(text):
    """
    Removes newline characters from a given text entry
    """
    return re.sub(r'\n', '', text)

In [5]:
# Tokenization
import nltk as nl
def tokenize(text):
    """
    Tokenizes a given text entry
    """
    tokens = nl.wordpunct_tokenize(text)
    return list(filter(lambda x: x.isalnum(), tokens))

In [6]:
# Removing stopwords OBS! Have to download the model first (spacy download en_core_web_sm)
import spacy as sp
en = sp.load('en_core_web_sm')
stopwords = en.Defaults.stop_words

def remove_stopwords(tokens):
    """
    Removes stopwords from a given list of tokens
    """
    return list(filter(lambda word: word not in stopwords, tokens))


In [7]:
# Stemming with spacy
import spacy as sp
nlp = sp.load('en_core_web_sm')
def lemmatize(tokens):
    """
    Stems a given list of tokens
    """
    doc = nlp(' '.join(tokens))

    lemmatized = [token.lemma_ for token in doc]
    return lemmatized

In [8]:
def tokenization_pipeline(text):
    """
    Applies the tokenization pipeline to a given text entry
    """
    return remove_stopwords(tokenize(remove_newlines(text.lower())))

In [9]:
ddf['tokens'] = ddf['content'].apply(tokenization_pipeline, meta=('tokens', 'object'))

### Split content into tokens by new-lines
Here we split the content into sentences with a set number of tokens. We do this as SBERT models only accept strings up to a certain length in terms of words.

In [5]:
TOKEN_LENGTH = 400

In [6]:
def recursive_token_split(text):
    """
    Recursively splits the text into tokens, such that none are more than 200 words long.
    :param text: The text to split.
    :return: A list of tokens.
    """
    # Count nr. of whitespaces
    nr_of_whitespaces = text.count(' ')

    # If there are more than 200 whitespaces, split the text in half
    if nr_of_whitespaces > TOKEN_LENGTH:
        split_index = len(text) // 2 
        return recursive_token_split(text[:split_index]) + recursive_token_split(text[split_index:])
    else:
        return text

In [7]:
import re
regex = re.compile(r'\n+')
def tokenize_by_newline(text, regex):
    """
    Tokenize the content by newlines. Also performs some additional checks as our model is not able to handle strings containing more than 256 words.
    :param text: The text to tokenize.
    :param regex: The regex to use for tokenization.
    :return: The tokenized text.
    """
    text = [t.strip() for t in regex.split(text)]
    text = [recursive_token_split(t) for t in text]
    
    return text

In [8]:
test = 'hello there \n        how \n are \n you'
tokenize_by_newline(test, regex)

['hello there', 'how', 'are', 'you']

In [9]:
# ddf_test = ddf.head(1000)
# ddf_test['tokenized_content'] = ddf_test['content'].apply(tokenize_by_newline, regex=regex)
# ddf_test.head(1).tokenized_content

0    [Life is an illusion, at least on a quantum le...
Name: tokenized_content, dtype: object

In [10]:
# first_entry = ddf_test.head(1).tokenized_content.values[0]
# first_entry

['Life is an illusion, at least on a quantum level, in a theory which has recently been confirmed by a set of researchers.',
 'They finally have the means to test John Wheeler’s delayed-choice theory and concluded that the physicist was right.',
 'In 1978, Mr Wheeler’s proposed experiment involved a moving object that was given the choice to act like a wave or a particle – the former acting as a vibration with a frequency that can distinguish it from other waves and the latter having no frequency that you can determine its position in space, unlike a wave – and at what point does it ‘decide’ to act like one or the other.',
 'At the time, the technology was not available to conduct a strong experiment, but scientists have now been able to carry it out.']

### Wordembedding

In [11]:
from sentence_transformers import SentenceTransformer
all_mpnet_base_v2 = SentenceTransformer('all-mpnet-base-v2', device='cuda')
all_MiniLM_L6_v2 = SentenceTransformer('all-MiniLM-L6-v2', device='cuda')

In [12]:
def batch(batch_size, individual_partition):
    """ 
    Collects the tokenized content of a batch in the partition, and encodes it with the sentence transformer. It keeps track of the original
    index of the content, so that it can be mapped back to the original dataframe.
    :param batch_size: The size of the batch.
    :param individual_partition: The partition to process (OBS: This process cannot be multithreaded).
    """
    # Collect the tokenized content of the batch
    batch_content = [individual_partition['tokenized_content'].values[i] for i in range(batch_size)]
    # Flatten the list
    batch_content = [item for sublist in batch_content for item in sublist]
    # Encode the content
    batch_embeddings = all_MiniLM_L6_v2.encode(batch_content, show_progress_bar=False, convert_to_numpy=True)
    # Collect the original index of the content
    batch_index = [individual_partition.index.values[i] for i in range(batch_size)]
    # Return the embeddings and the index
    return batch_embeddings, batch_index

In [60]:
def model_encode(token_vector, model):
    """" 
    Uses the given model to encode the given token vector
    """
    # Encode the tokens
    embeddings = model.encode(token_vector, show_progress_bar=True, batch_size=16)
    # Generate average of embeddings
    return embeddings.mean(axis=0)

In [14]:
# ddf_test = ddf.partitions[0]
# ddf_test['tokenized_content'] = ddf_test['content'].apply(tokenize_by_newline, regex=regex, meta=('tokenized_content', 'object'))
# ddf_test.head()

,id,content,type,tokenized_content
0,2,"Life is an illusion, at least on a quantum lev...",rumor,"[Life is an illusion, at least on a quantum le..."
1,6,"Unfortunately, he hasn’t yet attacked her for ...",hate,"[Unfortunately, he hasn’t yet attacked her for..."
2,7,The Los Angeles Police Department has been den...,hate,[The Los Angeles Police Department has been de...
3,8,The White House has decided to quietly withdra...,hate,[The White House has decided to quietly withdr...
4,9,“The time has come to cut off the tongues of t...,hate,[“The time has come to cut off the tongues of ...


In [15]:
# ddf_test['mpnet_embeddings'] = ddf_test['tokenized_content'].apply(model_encode, model=all_mpnet_base_v2, meta=('mpnet_embeddings', 'object'))
# ddf_test.head()

In [56]:
# Squish the entire tokenized content into a single list
# import numpy as np
# tokens = ddf_test['tokenized_content'].head(5000).values
# tokens

array([list(['Life is an illusion, at least on a quantum level, in a theory which has recently been confirmed by a set of researchers.', 'They finally have the means to test John Wheeler’s delayed-choice theory and concluded that the physicist was right.', 'In 1978, Mr Wheeler’s proposed experiment involved a moving object that was given the choice to act like a wave or a particle – the former acting as a vibration with a frequency that can distinguish it from other waves and the latter having no frequency that you can determine its position in space, unlike a wave – and at what point does it ‘decide’ to act like one or the other.', 'At the time, the technology was not available to conduct a strong experiment, but scientists have now been able to carry it out.']),
       list(['Unfortunately, he hasn’t yet attacked her for Islamic terrorism-connections, but we know he will. Below are several links that explain in detail why having Huma Abedin anywhere near the White House would be like

In [ ]:
BATCH_SIZE = 10
def compute_batches(daskdataframe):
    """ 
    Pulls out dask dataframe partitions, and computes the batches for each partition.
    """
    # Extract the desired rows
    ids = daskdataframe['id'].values
    values = daskdataframe['tokenized_content'].values
    types = daskdataframe['type'].values

    # Create a new pandas dataframe with the embeddings, id, type
    df = pd.DataFrame(columns=['id', 'type', 'embedding'])

    for i in range(len(ids) // BATCH_SIZE):

        # Squish the tokenized content into a single list (keep track of the original index)
        batch = [item for sublist in batch for item in sublist]

        # Run the model encoding
        batch_embeddings = all_MiniLM_L6_v2.encode(batch, show_progress_bar=True, convert_to_numpy=True)

        # Map the embeddings back to the original index, and construct the appropriate rows
        for j in range(len(batch_embeddings)):


    # Create a pandas dataframe with the embeddings, index and original values from the dask dataframe
    df = daskdataframe.head

    # return the embeddings



In [61]:
# tokens_flat = [item for sublist in tokens for item in sublist]
# tokens_flat

['Life is an illusion, at least on a quantum level, in a theory which has recently been confirmed by a set of researchers.',
 'They finally have the means to test John Wheeler’s delayed-choice theory and concluded that the physicist was right.',
 'In 1978, Mr Wheeler’s proposed experiment involved a moving object that was given the choice to act like a wave or a particle – the former acting as a vibration with a frequency that can distinguish it from other waves and the latter having no frequency that you can determine its position in space, unlike a wave – and at what point does it ‘decide’ to act like one or the other.',
 'At the time, the technology was not available to conduct a strong experiment, but scientists have now been able to carry it out.',
 'Unfortunately, he hasn’t yet attacked her for Islamic terrorism-connections, but we know he will. Below are several links that explain in detail why having Huma Abedin anywhere near the White House would be like inviting in the terror

In [58]:
# len(tokens_flat)

79892

In [62]:
# model_encode(tokens_flat, all_MiniLM_L6_v2)

Batches:   0%|          | 0/4994 [00:00<?, ?it/s]

array([-4.48858459e-03,  2.17180774e-02,  3.91423563e-03,  1.36059104e-02,
        1.91498995e-02,  1.14466082e-02,  1.09415362e-02, -9.18050483e-03,
       -3.69461183e-03,  2.88163836e-04, -4.19785909e-04,  2.45636795e-02,
       -8.31080193e-04, -3.33126122e-03, -1.10416580e-02,  8.74349941e-03,
       -9.31532308e-03, -1.12643512e-02, -1.65030174e-02,  9.06402897e-03,
       -2.31441557e-02,  8.61477945e-03,  2.06673611e-02,  1.42870229e-02,
       -4.03412268e-04,  7.81069649e-03, -8.87881778e-03, -8.81857332e-03,
       -1.14611834e-02, -5.91486879e-03,  3.81247955e-03,  5.98088931e-03,
       -2.48654373e-03,  9.31991264e-03,  5.26874978e-03,  1.03114909e-02,
        2.33270247e-02,  1.06190350e-02,  1.93507057e-02, -3.96903697e-03,
        1.42001230e-02, -3.71000022e-02,  7.20494892e-04,  4.53199307e-03,
       -1.02774287e-02, -4.03794274e-03, -9.36527364e-03, -1.23783771e-03,
        3.00234230e-03, -6.32032007e-03, -1.88652836e-02, -6.60433574e-03,
        3.77126597e-03, -

## Final data treatment pipeline

In [11]:
def process_data(df):
    """ 
    Applies the entire processing pipeline to the data.
    """
    # Tokenization
    df['tokens'] = df['content'].apply(tokenization_pipeline, meta=('tokens', 'object'))
    return df

## Split the dataset and write new parquet files

In [18]:
# Import the data
import dask.dataframe as dd
ddf = dd.read_parquet('data/filtered_parquets/', engine='pyarrow', compression='snappy')
ddf

,id,content,type,title,authors,domain,url
npartitions=900,,,,,,,
,int64,object,object,object,object,object,object
,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...
,...,...,...,...,...,...,...


In [19]:
# Apply the processing pipeline
ddf['tokens'] = ddf['content'].apply(tokenization_pipeline, meta=('tokens', 'object'))
ddf = ddf[['id', 'type', 'tokens']]
ddf.head()

,id,type,tokens
27,34,fake,"[headline, bitcoin, blockchain, searches, exce..."
28,35,fake,"[water, cooler, 1, 25, 18, open, thread, fake,..."
29,36,fake,"[veteran, commentator, calls, growing, ethnona..."
30,37,fake,"[lost, words, hidden, words, otters, banks, bo..."
31,38,fake,"[red, alert, bond, yields, screaming, inflatio..."


In [20]:
# Convert the tokens to a string (pyarrow not supporting lists)
ddf['tokens'] = ddf['tokens'].apply(lambda x: ','.join(x), meta=('tokens', 'object'))
ddf.head()

,id,type,tokens
27,34,fake,"headline,bitcoin,blockchain,searches,exceed,tr..."
28,35,fake,"water,cooler,1,25,18,open,thread,fake,news,cnn..."
29,36,fake,"veteran,commentator,calls,growing,ethnonationa..."
30,37,fake,"lost,words,hidden,words,otters,banks,books,rea..."
31,38,fake,"red,alert,bond,yields,screaming,inflation,comi..."


In [21]:
# Split the data into train and test
from dask_ml.model_selection import train_test_split
train, test = train_test_split(ddf, test_size=0.1, random_state=42)

# Write test and train to parquet files
train.to_parquet('data/train/', engine='pyarrow', compression='snappy')
test.to_parquet('data/test/', engine='pyarrow', compression='snappy')

/home/kristiandampedersen/.local/share/virtualenvs/fakenewsproject-Oj_5wi4E/lib/python3.10/site-packages/dask_ml/model_selection/_split.py:462: FutureWarning: The default value for 'shuffle' must be specified when splitting DataFrames. In the future DataFrames will automatically be shuffled within blocks prior to splitting. Specify 'shuffle=True' to adopt the future behavior now, or 'shuffle=False' to retain the previous behavior.
  warnings.warn(


Note: We split the dataset *before* we apply TF-IDF on it

In [2]:
# Read in train data
client.restart()
train = dd.read_parquet('data/train/', engine='pyarrow', compression='snappy')
train.head()

2023-03-16 15:28:56,043 - distributed.nanny - WARNING - Restarting worker
2023-03-16 15:28:56,076 - distributed.nanny - WARNING - Restarting worker
2023-03-16 15:28:56,079 - distributed.nanny - WARNING - Restarting worker
2023-03-16 15:28:56,085 - distributed.nanny - WARNING - Restarting worker


NameError: name 'dd' is not defined

We are now ready to run TF-IDF on it

### Dask hashing vectorizer 

In [15]:
from dask_ml.feature_extraction.text import HashingVectorizer
import numpy as np

vectorizer = HashingVectorizer(n_features=2**16, alternate_sign=False, ngram_range=(1, 2), analyzer='word', dtype=np.float32)
train_vectorized = vectorizer.fit_transform(train['tokens'])
train_vectorized

NameError: name 'train' is not defined